In [510]:
import math
import numpy as np
from datetime import date
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt

The Libor rate r(s) for maturity s years is given by the Nelson-Siegel model as:

$r(s) = B0 + B1*(tau/s)*(1-exp(-s/tau)) + B2*((tau/s)*(1-exp(-s/tau)) - exp(-s/tau))$

From this, we compute the discount factor $Z(0, s) = exp(-r(s) * s)$

In [511]:
def r(s, B0=0.0408, B1=-0.0396, B2=-0.0511, tau=1.614):
    if s == 0:
        return 0

    term_1 = B0
    term_2 = B1*(tau/s)*(1-np.exp(-s/tau))
    term_3 = B2*((tau/s)*(1-np.exp(-s/tau)) - np.exp(-s/tau))

    return term_1 + term_2 + term_3

In [512]:
def Z(s, B0=0.0408, B1=-0.0396, B2=-0.0511, tau=1.614):
    return np.exp(-r(s, B0, B1, B2, tau) * s)

### Calculation of survival probabilities Q(t)

In [513]:
def weekend_rollover(w_date):
    # If payment date falls on a weekday, roll it over to Monday.
    if w_date.weekday() == 5:
        w_date += relativedelta(days=2)
    elif w_date.weekday() == 6:
        w_date += relativedelta(days=1)

    return w_date

In [552]:
payment_period = 0.25 # 4 payments per year

# maturity dates for the known spreads, as calculated on February 28, 2014 
spread_maturity_dates_before_adjustment = [
    date(2014, 9, 20),
    date(2015, 3, 20),
    date(2016, 3, 20),
    date(2017, 3, 20),
    date(2018, 3, 20),
    date(2019, 3, 20),
    date(2021, 3, 20),
    date(2024, 3, 20),
    date(2034, 3, 20),
    date(2044, 3, 20)
]

spread_maturity_dates = [ weekend_rollover(mat_date) for mat_date in spread_maturity_dates_before_adjustment ]
# spread data: {time(in years): spread (bps)}
spread_data = {
    0.5: 103.07,
    1: 104.68,
    2: 106.74,
    3: 110.31,
    4: 113.38,
    5: 116.98,
    7: 128.78,
    10: 143.51,
    20: 159.43,
    30: 164.13
}

Payment convention assumed is (Actual / 360).

If payment dates fall on weekend, they are rolled over to Monday.

h(s), which is the instantaneous forward default rate, is assumed to be piecewise constant between the different maturities for which we know the spreads.

If we can calibrate the values of this instantaneous forward default rate, we can get the survival curve.

$-\ln(Q(t)) = \int_0^t h(s) \,ds$

In [543]:
# Payment dates for standard CDS = 20 March, 20 June, 20 September, 20 December
standard_cds_payment_dates = [date(2014, 3, 20), date(2014, 6, 20), date(2014, 9, 20), date(2014, 12, 20)] # year is irrelevant

def next_payment_date(current_date, payment_dates):
    payment_date = current_date

    if current_date.month < payment_dates[0].month:
        payment_date = date(current_date.year, payment_dates[0].month, payment_dates[0].day)
    elif current_date.month > payment_dates[0].month and current_date.month < payment_dates[1].month:
        payment_date = date(current_date.year, payment_dates[1].month, payment_dates[1].day)
    elif current_date.month > payment_dates[1].month and current_date.month < payment_dates[2].month:
        payment_date = date(current_date.year, payment_dates[2].month, payment_dates[2].day)
    elif current_date.month > payment_dates[2].month and current_date.month < payment_dates[3].month:
        payment_date = date(current_date.year, payment_dates[3].month, payment_dates[3].day)
    else:
        # date belongs to one of the payment months
        if current_date.day < payment_dates[0].day:
            payment_date = date(current_date.year, current_date.month, payment_dates[0].day)
        else:
            payment_date = next_payment_date(current_date + relativedelta(months=1), payment_dates)

    # If payment date falls on a weekday, roll it over to Monday.
    payment_date = weekend_rollover(payment_date)

    return payment_date

In [516]:
contract_date = date(2014, 2, 28)
effective_date = contract_date + relativedelta(days=1)

In [517]:
def K(T_i, m=12):
    return math.ceil(m*T_i)

def s(k, T_i, m=12):
    # We are choosing m=12, because we want to calculate the value in monthly subintervals.
    return k*T_i / K(T_i, m)

In [518]:
def Q_h(t, h_i, prev_h={}):
    # t: time in years
    # h_i: instantaneous forward default rate for ith maturity

    end_date = effective_date + relativedelta(days=int(t*360))

    h_t = 0
    for idx, mat_date in enumerate(spread_maturity_dates):
        start_date = effective_date if idx==0 else spread_maturity_dates[idx-1]

        if mat_date < end_date:
            h_t += prev_h[mat_date] * ((mat_date - start_date).days / 360)
        elif mat_date >= end_date:
            h_t += h_i * ((end_date - start_date).days / 360)

            break
    
    return np.exp(-h_t)

# differential of Q_h
def diff_Q_h(t, h_i, prev_h):
    end_date = effective_date + relativedelta(days=int(t*360))
    diff = 0

    for idx, mat_date in enumerate(spread_maturity_dates):
        if mat_date >= end_date:
            start_date = effective_date if idx == 0 else spread_maturity_dates[idx-1]
            diff = (end_date - start_date).days / 360
            break

    return -diff*Q_h(t, h_i, prev_h)

In [554]:

def standard_cds_mtm_value(h_i, maturity_date, prev_h={}, payment_dates=standard_cds_payment_dates, R=0.45):
    a = (1-R) * 100/2 # Multiplying by 100 to convert % to bps
    b = 0

    T = (maturity_date - effective_date).days / 360
    
    for k in range(K(T)):
        b += (Z(s(k, T)) + Z(s(k+1, T))) * (Q_h(s(k,T), h_i, prev_h) - Q_h(s(k+1, T), h_i, prev_h))

    maturity_date_index = spread_maturity_dates.index(maturity_date)
    
    c = 0.5 * list(spread_data.values())[maturity_date_index]
    d = 0

    current_date = effective_date
    next_pay_date = next_payment_date(effective_date, payment_dates=payment_dates)

    while next_pay_date <= maturity_date:
        delta_t = ((next_pay_date - current_date).days) / 360 # day-counting convention
        t_1 = (current_date - effective_date).days / 360
        t_2 = (next_pay_date - effective_date).days / 360

        d += delta_t * Z(t_2) * (Q_h(t_1, h_i, prev_h) + Q_h(t_2, h_i, prev_h))
        
        current_date = next_pay_date
        next_pay_date = next_payment_date(next_pay_date, payment_dates=payment_dates)
    
    return a*b-c*d

In [555]:
# differential of cds MTM value, for using in Newton-Raphson method.
def d_standard_cds_mtm_value(h_i, maturity_date, prev_h={}, payment_dates=standard_cds_payment_dates, R=0.45):
    a = (1-R) * 100/2 # Multiplying by 100 to convert % to bps
    b = 0
    
    T = (maturity_date - effective_date).days / 360

    for k in range(K(T)):
        b += (Z(s(k, T)) + Z(s(k+1, T))) * (diff_Q_h(s(k, T), h_i, prev_h) - diff_Q_h(s(k+1, T), h_i, prev_h))

    maturity_date_index = spread_maturity_dates.index(maturity_date)
    c = 0.5 * list(spread_data.values())[maturity_date_index]
    d = 0

    current_date = effective_date
    next_pay_date = next_payment_date(effective_date, payment_dates=payment_dates)

    while next_pay_date <= maturity_date:
        delta_t = ((next_pay_date - current_date).days) / 360 # day-counting convention
        t_1 = (current_date - effective_date).days / 360
        t_2 = (next_pay_date - effective_date).days / 360

        d += delta_t * Z(t_2) * (diff_Q_h(t_1, h_i, prev_h) + diff_Q_h(t_2, h_i, prev_h))
        
        current_date = next_pay_date
        next_pay_date = next_payment_date(next_pay_date, payment_dates=payment_dates)
    
    return a*b-c*d

In [556]:
d_standard_cds_mtm_value(0.01, spread_maturity_dates[0])

47.77721806748909

In [522]:
def newton_raphson_method(f, d_f, maturity_date, prev_h={}, x=0.01, eps=1e-6):
    np.seterr(all="raise")

    val = f(x, maturity_date, prev_h)
    while abs(val) > eps:
        diff = d_f(x, maturity_date, prev_h) #differential of f(h1)

        if np.abs(diff) > np.sqrt(np.finfo(float).eps):            
            x = x - float(val) / diff
            val = f(x, maturity_date, prev_h)
        else:
            return None
        
    return x

In [557]:
h_values = {}

for mat_date in spread_maturity_dates:
    h = newton_raphson_method(standard_cds_mtm_value, d_standard_cds_mtm_value, mat_date, h_values)
    h_values[mat_date] = h
    
h_values

{datetime.date(2014, 9, 22): 1.906293407117043,
 datetime.date(2015, 3, 20): 1.953364459804524,
 datetime.date(2016, 3, 21): 2.0157485147215066,
 datetime.date(2017, 3, 20): 2.129561259497372,
 datetime.date(2018, 3, 20): 2.2270680733742383,
 datetime.date(2019, 3, 20): 2.3402157994365926,
 datetime.date(2021, 3, 22): 2.7012830118133793,
 datetime.date(2024, 3, 20): 3.1345861935068893,
 datetime.date(2034, 3, 20): 3.5864503288466683,
 datetime.date(2044, 3, 21): 3.717295971213892}

In [558]:
def Q(dat):
    # Above we have a function Q_h(t, h_i, prev_h={}) which requires t and h_i as arguments.
    # Now using the values of h we have found, this function provides value of Q at any date t years from the effective date.
    if dat < effective_date:
        return 0
    
    t = (dat - effective_date).days / 360
    
    for mat_date in spread_maturity_dates:
        if dat <= mat_date:
            return Q_h(t, h_values[mat_date], h_values)
        
    return Q_h(t, list(h_values.values())[-1], h_values) # if given time is more than the max date we have, return last h value

In [559]:
Q_values = {}

for mat_date in spread_maturity_dates:
    Q_values[mat_date] = Q(mat_date)
    
Q_values

{datetime.date(2014, 9, 22): 0.3377233566214906,
 datetime.date(2015, 3, 20): 0.12786446954790137,
 datetime.date(2016, 3, 21): 0.016379445843662035,
 datetime.date(2017, 3, 20): 0.0019018064455564834,
 datetime.date(2018, 3, 20): 0.0002000857851023297,
 datetime.date(2019, 3, 20): 1.853834154334645e-05,
 datetime.date(2021, 3, 22): 7.575345300809694e-08,
 datetime.date(2024, 3, 20): 5.5749722277052555e-12,
 datetime.date(2034, 3, 20): 8.744242869471106e-28,
 datetime.date(2044, 3, 21): 3.593774008363481e-44}

In [560]:
for mat_date in spread_maturity_dates:
    print(standard_cds_mtm_value(h_values[mat_date],mat_date,h_values))

-9.64206492426456e-12
-6.750155989720952e-13
-5.071854047855595e-11
-4.345679371908773e-11
-9.99733629214461e-11
-2.951736632894608e-10
-6.3358882584907406e-09
-1.2342816546606628e-07
-2.1316282072803006e-14
-1.1368683772161603e-13


### Pricing non-standard CDS using the calibrated survival curve

In [583]:
# Payment dates for non-standard CDS = 15 February, 15 May, 15 August, 15 November
nonstandard_cds_payment_dates = [date(2014, 2, 15), date(2014, 5, 15), date(2014, 8, 15), date(2014, 11, 15)] # year is irrelevant
ns_effective_date = date(2014, 3, 1)

In [587]:
def nonstandard_cds_mtm_value(maturity_date, spread=170, payment_dates=nonstandard_cds_payment_dates, R=0.6):
    a = (1-R) * 100/2 # Multiplying by 100 to convert % to bps
    b = 0

    T = (maturity_date - ns_effective_date).days / 360
    
    for k in range(K(T)):
        date_s_k = ns_effective_date + relativedelta(days=s(k, T)*360)
        date_s_k1 = ns_effective_date + relativedelta(days=s(k+1, T)*360)

        b += (Z(s(k, T)) + Z(s(k+1, T))) * (Q(date_s_k) - Q(date_s_k1))

    c = 0.5 * spread
    d = 0

    current_date = ns_effective_date
    next_pay_date = next_payment_date(ns_effective_date, payment_dates=payment_dates)

    while next_pay_date <= maturity_date:
        delta_t = ((next_pay_date - current_date).days) / 360 # day-counting convention
        t_1 = (current_date - ns_effective_date).days / 360
        t_2 = (next_pay_date - ns_effective_date).days / 360

        d += delta_t * Z(t_2) * (Q(current_date) + Q(next_pay_date))
        
        current_date = next_pay_date
        next_pay_date = next_payment_date(next_pay_date, payment_dates=payment_dates)
    
    return a*b-c*d

In [588]:
nonstandard_cds_mtm_value(date(2042, 5, 15))

-49.535347403931794